In [1]:
import pandas as pd
import os
import numpy as np
import polars as pl

In [2]:
def get_competition_terms():
    train_terms_df = pl.read_csv('../input/cafa-5-protein-function-prediction/Train/train_terms.tsv', 
                separator='\t', new_columns=['id', 'term','ont'])
    return list(train_terms_df['term'].unique())

targets = get_competition_terms()
print(len(targets))

31466


In [18]:
targets

['GO:0071406',
 'GO:0010928',
 'GO:1902178',
 'GO:1990169',
 'GO:0097242',
 'GO:0006206',
 'GO:0030207',
 'GO:0051377',
 'GO:0032187',
 'GO:0042821',
 'GO:0046860',
 'GO:0047395',
 'GO:1900457',
 'GO:0022402',
 'GO:0060532',
 'GO:0008771',
 'GO:0050853',
 'GO:2000659',
 'GO:0005874',
 'GO:0033344',
 'GO:0046436',
 'GO:0051722',
 'GO:0019233',
 'GO:0021961',
 'GO:0061514',
 'GO:0070966',
 'GO:1990111',
 'GO:0010961',
 'GO:0015169',
 'GO:0015990',
 'GO:1901989',
 'GO:0009528',
 'GO:0110043',
 'GO:0097655',
 'GO:0004078',
 'GO:0015112',
 'GO:0061421',
 'GO:0035850',
 'GO:0043232',
 'GO:1902086',
 'GO:0016893',
 'GO:0140439',
 'GO:1900584',
 'GO:0033293',
 'GO:2000706',
 'GO:0080017',
 'GO:0004802',
 'GO:0043951',
 'GO:0035866',
 'GO:0043772',
 'GO:1990984',
 'GO:0010651',
 'GO:0004420',
 'GO:0003186',
 'GO:0060827',
 'GO:0006950',
 'GO:1901315',
 'GO:1990962',
 'GO:0033560',
 'GO:0000992',
 'GO:0000993',
 'GO:0047352',
 'GO:2000165',
 'GO:0097150',
 'GO:0006497',
 'GO:0099139',
 'GO:00156

In [3]:
def get_competition_test_ids():
    sample_sub = pl.read_csv('../input/cafa-5-protein-function-prediction/sample_submission.tsv', 
                separator='\t', new_columns=['id', 'term','pred'])
    return list(sample_sub['id'].unique())
ids = get_competition_test_ids()
print(len(ids))

141864


## Concatenate DQZ ontology submissions

In [6]:
suffix = '_zero_10'
th = 0.5

In [9]:
bp_fn = f'../output/DGZ/bp_th-0.1_deepgozero{suffix}_submission.tsv'
mf_fn = f'../output/DGZ/mf_th-0.1_deepgozero{suffix}_submission.tsv'
cc_fn = f'../output/DGZ/cc_th-0.1_deepgozero{suffix}_submission.tsv'

In [25]:
from tqdm.auto import tqdm

all_dicts = {id:{} for id in ids}

for fn in tqdm([bp_fn, mf_fn, cc_fn]):
    with open(fn, 'r') as file:
        while True:
            # Get next line from file
            line = file.readline()

            # if line is empty
            # end of file is reached
            if not line:
                break

            try:
                parts = line.strip().split('\t')
                if len(parts) < 3: continue
                _id, _term, _p = parts
                _p = float(_p)

                if _p < th: continue
                if _id not in all_dicts: continue
                if _term not in targets: continue

            except:
                break
            prev = all_dicts[_id].get(_term, 0)
            all_dicts[_id][_term] = max(prev, _p)

  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
with open(f'../output/DGZ/th-{th}_deepgozero{suffix}_submission.tsv', 'w') as file:
    for _id in tqdm(ids, total=len(ids)):
        all_terms = list(all_dicts[_id].keys())
        for term in all_terms:
            file.write(f'{_id}\t{term}\t{all_dicts[_id][term]:.3f}\n')

  0%|          | 0/141864 [00:00<?, ?it/s]